# Import library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/1AfDhmrh6yjrRAOYL36wIoH14ei6QNXHjKIWyYE2kOLjnENYtIRMQYn2z2lM


In [ ]:
! nvidia-smi

Fri Oct 30 07:15:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import time
import random
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
print("tf version:", tf.__version__)

tf version: 2.3.0


# Set random seed

In [ ]:
os.environ['PYTHONHASHSEED']=str(1)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

# Limit GPU resource

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


# Plot acc and loss figure

In [ ]:
def plot_acc(train_history):
    """
    This function is used for plot accuracy curve
    during training process
    Parameters
    -----------
    train_history : variables names of training process
    """
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Train History')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['train', 'validation'], loc = 'upper left')

In [ ]:
def plot_loss(train_history):
    """
    This function is used for plot loss figure
    during training process
    Parameters
    -----------
    train_history : variables names of training process
    """
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Train History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['train', 'validation'], loc = 'upper left')

# plot confusion matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix',
                          cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    Parameter
    ---------
    cm : confusion matrix
    classes : various category
    normalize : True, False
    title : you can type anthing you in it, but the data type must be "string"
    default 'Confusion matrix'
    cmap : default 'plt.cm.Blues'
    reference : https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    else:
        print('Confusion matrix, without normalization')


    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# One hot encoder to label encoder

In [ ]:
def onehot_to_label(data):
    """
    This function is used for transform one hot encoding
    to label encoding
    
    Paramenters
    -----------
    data : array-like of shape (n_samples, n_category)
    
    Returns
    -------
    label encoder
    """
    label = []
    for i in range(len(data)):
        for j in range(len(data[i])):
            if data[i][j] == 1:
                label.append(j)
    return label

# Load data intra subject

In [ ]:
def load_data(time, condition):
    
    """
    This function is used for load pickle file
    from same folder
    
    Parameters
    ----------
    time : '2s', '1s', '0.5s'
    condition : 'stationary', 'holding'
    
    Returns
    -------
    data : tensor-like of shape (number_of_samples, time_length, 8, 8)
    label : label of each category
    """
    
    
    labels = []
    gesture_type = 1
    gestures = ['pat', 'stroke', 'grab', 'poke', 'scratch', 'notouch']
    pickle_path = '/content/drive/My Drive/研究資料(共用)/pickle/' + condition + ' test'
    if time == '2s':
        data = np.zeros((1,120, 8, 8))
    elif time == '1s':
        data = np.zeros((1,60, 8, 8))
    else:
        data = np.zeros((1,30, 8, 8))
    
    if condition == 'stationary':
        for gesture in gestures:
            file_name = 'raw_data_' + gesture + '_' + time + '_s1s25_s.pck1'
            pickle_file = open(os.path.join(pickle_path, file_name), 'rb')
            array = pickle.load(pickle_file)
            pickle_file.close()
            data = np.concatenate((data, array))
            label = [gesture_type for i in range(len(array))]
            labels = np.concatenate((labels, label))
            gesture_type += 1
    else:
        for gesture in gestures:
            file_name = 'raw_data_' + gesture + '_' + time + '_s1s25_h.pck1'
            pickle_file = open(os.path.join(pickle_path, file_name), 'rb')
            array = pickle.load(pickle_file)
            pickle_file.close()
            data = np.concatenate((data, array))
            label = [gesture_type for i in range(len(array))]
            labels = np.concatenate((labels, label))
            gesture_type += 1

    o_label = np.array(pd.get_dummies(labels))
    data = data[1:]
    data = data.astype('float32')
    return data, o_label

# load data inter subject

In [ ]:
def load_random_subject(time, condition_1, condition_2, folder):
    """
    This function is used for load pickle file
    from various folder
    
    Parameters
    ----------
    time : '2s', '1s', '0.5s'
    condition_1 : 'stationary', 'holding'
    condition_2 : 'train', 'test'
    folder : 1, 2, 3, ...10
    
    Returns
    -------
    data : tensor-like of shape (number_of_samples, time_length, 8, 8)
    label : label of each category
    """
    labels = []
    gestures = ['pat', 'stroke', 'grab', 'poke', 'scratch', 'notouch']
    gesture_type = 1
    pickle_path = '/content/drive/My Drive/研究資料(共用)/different subject train test/' + condition_1 + '_s1s25' + '/cv_' + str(folder)
    if time == '2s':
        data = np.zeros((1,120, 8, 8))
    elif time == '1s':
        data = np.zeros((1,60, 8, 8))
    else:
        data = np.zeros((1,30, 8, 8))
    
    if condition_1 == 'stationary':
        for gesture in gestures:
            file_name = 'raw_data_' + condition_2 + '_' + gesture + '_' + time + '_s1s25_s_cv_' + str(folder) + '.pck1'
            pickle_file = open(os.path.join(pickle_path, file_name), 'rb')
            array = pickle.load(pickle_file)
            pickle_file.close()
            data = np.concatenate((data, array))
            label = [gesture_type for i in range(len(array))]
            labels = np.concatenate((labels, label))
            gesture_type += 1
    else:
        for gesture in gestures:
            file_name = 'raw_data_' + condition_2 + '_' + gesture + '_' + time + '_s1s25_h_cv_' + str(folder) + '.pck1'
            pickle_file = open(os.path.join(pickle_path, file_name), 'rb')
            array = pickle.load(pickle_file)
            pickle_file.close()
            data = np.concatenate((data, array))
            label = [gesture_type for i in range(len(array))]
            labels = np.concatenate((labels, label))
            gesture_type += 1
               
    o_label = np.array(pd.get_dummies(labels))
    data = data[1:]
    data = data.astype('float32')
    return data, o_label

# Random select data

In [ ]:
def random_select(s_data, s_olabel, h_data, h_olabel):
    """
    This function can shuffle the data
    
    Parameter
    ---------
    s_data: data from stationary dataset
    s_olabel : label from stationary dataset
    h_data: data from holding dataset
    h_olabel : label from holding dataset
    
    Returns
    -------
    half of numbers of each posture dataset
    """
    len_s = len(s_olabel)
    len_h = len(h_olabel)
    s_randomize = np.arange(len_s)
    h_randomize = np.arange(len_h)
    np.random.shuffle(s_randomize)
    np.random.shuffle(h_randomize)
    half_s_data = s_data[s_randomize[:int(0.5 * len_s)]]
    half_h_data = h_data[h_randomize[:int(0.5 * len_h)]]
    half_s_olabel = s_olabel[s_randomize[:int(0.5 * len_s)]]
    half_h_olabel = h_olabel[h_randomize[:int(0.5 * len_h)]]
    
    return half_s_data, half_s_olabel, half_h_data, half_h_olabel

# Identity Block

In [ ]:
def identity_block(X, f, filters):
    """
    This function is used for build the fundamental residual block
    
    Parameter
    ---------
    X : input data
    f : filter size
    filters: numbers of filters
    
    Returns
    -------
    tensor-like shape
    """
    # Retrive Filters
    f1, f2, f3 = filters
    # Save input value
    x_shortcut = X
    
    # first component of main path
    
    x = tf.keras.layers.Conv2D(f1, kernel_size = 1, padding = 'same', kernel_initializer = 'he_uniform',
                              data_format = 'channels_first')(X)
    x = tf.keras.layers.BatchNormalization(axis = 1)(x)
    x = tf.keras.activations.swish(x)
    
    # second component of main path
    
    x = tf.keras.layers.Conv2D(f2, kernel_size = f, padding = 'same', kernel_initializer = 'he_uniform',
                              data_format = 'channels_first')(x)
    x = tf.keras.layers.BatchNormalization(axis = 1)(x)
    x = tf.keras.activations.swish(x)
    
    # Third component of main path
    
    x = tf.keras.layers.Conv2D(f3, kernel_size = 1, padding = 'same', kernel_initializer = 'he_uniform',
                              data_format = 'channels_first')(x)
    x = tf.keras.layers.BatchNormalization(axis = 1)(x)
    
    # Final step: Add shortcut value to main path, and pass it through a Swish activation
    x = tf.keras.layers.add([x, x_shortcut])
    x = tf.keras.activations.swish(x)
    
    return x

# 2D CNN with BatchNormalization

In [ ]:
def cnn2d_bn(x, f, size):
    """
    This function is used for build the fundamental convolutional block
    
    Parameter
    ---------
    X : input data
    size : filter size
    filters: numbers of filters
    
    Returns
    -------
    tensor-like shape
    """
    
    cnn2d = tf.keras.layers.Conv2D(f, kernel_size = size, padding = 'same',
                                  kernel_initializer = 'he_uniform',
                                  data_format = 'channels_first')(x)
    
    bn = tf.keras.layers.BatchNormalization(axis = 1)(cnn2d)
    
    act = tf.keras.activations.swish(bn)
    
    return act

# Model

In [ ]:
def cnn_2d(input_shape, learning_rate):
    """
    This function is used for build the model
    
    Parameter
    ---------
    input_shape : define the input shape
    learning_rate : define learning rate which was set in optimizer
    
    Returns
    -------
    whole model
    """
    
    input_data = tf.keras.Input(shape = input_shape, name = 'input_data_shape')
    
    x = cnn2d_bn(input_data, 32, 1)
    
    x = identity_block(x, (3, 3), [8, 8, 32])
    
    x = tf.keras.layers.MaxPool2D(pool_size = (2, 2), data_format = 'channels_first')(x)
    
    x = identity_block(x, (3, 3), [16, 16, 32])
    
    x = cnn2d_bn(x, 64, 1)
    
    x = tf.keras.layers.GlobalAveragePooling2D(data_format = 'channels_first')(x)
    
    output = tf.keras.layers.Dense(6, activation = 'softmax')(x)
    
    model = tf.keras.Model(inputs = input_data, outputs = output)
    
    ranger = tfa.optimizers.Lookahead(tfa.optimizers.RectifiedAdam(lr = learning_rate, warmup_proportion = 0.1), 
                                      sync_period = 6, slow_step_size = 0.6)
    model.compile(optimizer = ranger, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
cnn_2d((120, 8, 8)).summary()

# intra subject (same posture)

In [ ]:
# Theses list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
phase = ['stationary', 'holding']
#--------------------------------------------------------------------------------
# this part is to define input data shape and load data
#--------------------------------------------------------------------------------
for c in phase:
    conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
    for length in times:
        i = 1
        cvscores = []
        print('Now start 10 fold cross validation')
        print('phase = ' + c)
        print('data_length = ' + length)
        print('Now, start for ' + length)
        if length == '2s' :
            input_shape = (120, 8, 8)
        elif length == '1s':
            input_shape = (60, 8, 8)
        else:
            input_shape = (30, 8, 8)
        data, o_label = load_data(length, c)
        label = onehot_to_label(o_label)
        label = np.reshape(label, (len(label), 1))
        k_fold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1)
#--------------------------------------------------------------------------------
# this part is to perform 10-fold cross validation
#--------------------------------------------------------------------------------
        for train, validate in k_fold.split(data, label):
            print('Now, start the cv: ',i)
            x_train, y_train = data[train], label[train]
            x_validate, y_validate = data[validate], label[validate]
            y_train = tf.keras.utils.to_categorical(y_train, num_classes = 6)
            y_val = tf.keras.utils.to_categorical(y_validate, num_classes = 6)
            model = cnn_2d(input_shape = input_shape, learning_rate = 1e-3)
            history = model.fit(x_train, y_train, epochs = 35, batch_size = 32, verbose = 2, shuffle = True,
                               validation_data = (x_validate, y_val))
            pred = np.argmax(model.predict(x_validate), axis = 1)
            real_label = onehot_to_label(y_val)
            scores = accuracy_score(real_label, pred)* 100
            print("The accuracy in this fold: %f"%scores)
            print("=============================")
            cvscores.append(scores)
            conf_mat = np.expand_dims(confusion_matrix(y_validate, pred), axis = 2)
            conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
            i += 1
#--------------------------------------------------------------------------------
# this part is to plot acc,loss curve and print out the average accuracy
#--------------------------------------------------------------------------------    
            # plot_acc(history)
            # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + c + '_' + length + '_intra subject accuracy.png')
            # plt.show()
            # plot_loss(history)
            # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + c + '_' + length + '_intra subject loss.png')
            # plt.show()
        print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
        print("=============================================")
        print("=============================================")
#--------------------------------------------------------------------------------
# this part i to plot confusion matrix
#--------------------------------------------------------------------------------        
    conf_mat_sum = np.sum(conf_mat_all, axis = 2)
    plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                      normalize = True)
    # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/confusion matrix/' + '2D_CNN' + '_' + c + '_intra subject confusion_matrix.png')
    plt.show()
#--------------------------------------------------------------------------------

Now start 10 fold cross validation
phase = stationary
data_length = 2s
Now, start for 2s


FileNotFoundError: ignored

# intra subject (different posture)

In [ ]:
# These list can be iterated throgh for loop
times = ['2s', '1s', '0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
# this part is to define input data shape and confusion matrix, then load data
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    cvscores = []
    print('Now, start for ' + length)
    if length == '2s' :
        input_shape = (120, 8, 8)
    elif length == '1s':
        input_shape = (60, 8, 8)
    else:
        input_shape = (30, 8, 8)    
    s_data, s_o_label = load_data(length, 'stationary')
    h_data, h_o_label = load_data(length, 'holding')
#--------------------------------------------------------------------------------
# this part is to perform 10-fold cross validation
#--------------------------------------------------------------------------------
    for i in range(1, 11):
        print('Now, start the cv: ',i)
        x_s_train, x_s_test, y_s_train, y_s_test = train_test_split(s_data, s_o_label, test_size = 0.1, shuffle = True)
        x_h_train, x_h_test, y_h_train, y_h_test = train_test_split(h_data, h_o_label, test_size = 0.1, shuffle = True)
        model = cnn_2d(input_shape = input_shape, learning_rate = 1e-3)
        history = model.fit(x_h_train, y_h_train, epochs = 20, batch_size = 32, verbose = 0,
                            shuffle = True, validation_data = (x_s_test, y_s_test))
        pred = np.argmax(model.predict(x_s_test), axis = 1)
        real_label = onehot_to_label(y_s_test)
        scores = accuracy_score(real_label, pred) * 100
        print('The accuracy in this fold is: %f'%scores)
        conf_mat = np.expand_dims(confusion_matrix(real_label, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat, conf_mat_all], axis = 2)
#--------------------------------------------------------------------------------
# this part is to plot acc and loss curve and print out the average accuracy
#--------------------------------------------------------------------------------   
        # plot_acc(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_intra subject train_holding_test_stationary accuracy.png')
        # plt.show()
        # plot_loss(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_intra subject train_holding_test_stationary loss.png')
        # plt.show()
        cvscores.append(scores)
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#--------------------------------------------------------------------------------
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues, normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/confusion matrix/' + '2D_CNN_' + '_train_holding_test_stationary confusion_matrix.png')
plt.show()

In [ ]:
# These list can be iterated throgh for loop
times = ['2s', '1s', '0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
# this part is to define input data shape and confusion matrix, then load data
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    cvscores = []
    print('Now, start for ' + length)
    if length == '2s' :
        input_shape = (120, 8, 8)
    elif length == '1s':
        input_shape = (60, 8, 8)
    else:
        input_shape = (30, 8, 8)    
    s_data, s_o_label = load_data(length, 'stationary')
    h_data, h_o_label = load_data(length, 'holding')
#--------------------------------------------------------------------------------
# this part is to perform 10-fold cross validation
#--------------------------------------------------------------------------------
    for i in range(1, 11):
        print('Now, start the cv: ',i)
        x_s_train, x_s_test, y_s_train, y_s_test = train_test_split(s_data, s_o_label, test_size = 0.1, shuffle = True)
        x_h_train, x_h_test, y_h_train, y_h_test = train_test_split(h_data, h_o_label, test_size = 0.1, shuffle = True)
        model = cnn_2d(input_shape = input_shape, learning_rate = 1e-3)
        history = model.fit(x_s_train, y_s_train, epochs = 20, batch_size = 32, verbose = 2,
                            shuffle = True, validation_data = (x_h_test, y_h_test))
        pred = np.argmax(model.predict(x_h_test), axis = 1)
        real_label = onehot_to_label(y_h_test)
        scores = accuracy_score(real_label, pred) * 100
        print('The accuracy is this fold is : %f'%scores)
        cvscores.append(scores)
        conf_mat = np.expand_dims(confusion_matrix(real_label, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat, conf_mat_all], axis = 2)
#--------------------------------------------------------------------------------
# this part is to plot acc, loss curve and print out the average accuracy
#--------------------------------------------------------------------------------
        # plot_acc(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_intra subject train_stationary_test_holding accuracy.png')
        # plt.show()
        # plot_loss(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_intra subject train_stationary_test_holding loss.png')
        # plt.show()
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#--------------------------------------------------------------------------------
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues, normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/confusion matrix/' + '2D_CNN_' + '_train_stationary_test_holding confusion_matrix.png')
plt.show()
#--------------------------------------------------------------------------------

# Inter subject (same posture)

In [ ]:
# These list can be iterated throgh for loop
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
# this part is to define input data shape and confusion matrix
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    cvscores = []
    print('Now, start for ' + length)
    if length == '2s' :
        input_shape = (120, 8, 8)
    elif length == '1s':
        input_shape = (60, 8, 8)
    else:
        input_shape = (30, 8, 8)
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------     
    for i in range(1,11):
        train_data, train_olabel = load_random_subject(length, 'stationary', 'train', i)
        test_data, test_olabel = load_random_subject(length, 'stationary', 'test', i) 
        print('Now, start the cv: ',i)
        model = cnn_2d(input_shape = input_shape, learning_rate = 1e-3)
        history = model.fit(train_data,train_olabel,epochs = 20, batch_size = 32,verbose = 2,shuffle = True,
                          validation_data = (test_data,test_olabel))
        pred = np.argmax(model.predict(test_data), axis = 1)
        real_label = onehot_to_label(test_olabel)
        scores = accuracy_score(real_label, pred) * 100
        print("The accuracy in this fold: %f"%scores)
        print("=============================")
        cvscores.append(scores)
        conf_mat = np.expand_dims(confusion_matrix(real_label, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
#--------------------------------------------------------------------------------
# this part is to plot acc, loss curve and print out the average accuracy
#--------------------------------------------------------------------------------
        # plot_acc(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (stationary) accuracy.png')
        # plt.show()
        # plot_loss(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (stationary) loss.png')
        # plt.show()
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#--------------------------------------------------------------------------------    
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                  normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/confusion matrix/' + '2D_CNN' + ' stationary '+ ' inter_subject confusion_matrix.png')
plt.show()
#--------------------------------------------------------------------------------

In [ ]:
# These list can be iterated throgh for loop
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
#--------------------------------------------------------------------------------
# this part is to define input data shape and confusion matrix
#--------------------------------------------------------------------------------
for length in times:
    cvscores = []
    print('Now, start for ' + length)
    if length == '2s' :
        input_shape = (120, 8, 8)
    elif length == '1s':
        input_shape = (60, 8, 8)
    else:
        input_shape = (30, 8, 8)
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#-------------------------------------------------------------------------------- 
    for i in range(1,11):
        train_data, train_olabel = load_random_subject(length, 'holding', 'train', i)
        test_data, test_olabel = load_random_subject(length, 'holding', 'test', i) 
        print('Now, start the cv: ',i)
        model = cnn_2d(input_shape = input_shape, learning_rate = 1e-3)
        history = model.fit(train_data,train_olabel,epochs = 20, batch_size = 32,verbose = 2,shuffle = True,
                          validation_data=(test_data,test_olabel))
        pred = np.argmax(model.predict(test_data), axis = 1)
        real_label = onehot_to_label(test_olabel)
        scores = accuracy_score(real_label, pred) * 100
        print("The accuracy in this fold: %f"%scores)
        print("=============================")
        cvscores.append(scores)
        conf_mat = np.expand_dims(confusion_matrix(real_label, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
#--------------------------------------------------------------------------------
# this part is to plot acc, loss curve and print out the average accuracy
#--------------------------------------------------------------------------------
        # plot_acc(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (holding) accuracy.png')
        # plt.show()
        # plot_loss(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (holding) loss.png')
        # plt.show()
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
# -------------------------------------------------------------------------------   
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                  normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/confusion matrix/' + '2D_CNN' + ' holding '+ ' inter_subject(holding) confusion_matrix.png')
plt.show()

# Inter subject (different posture)

In [ ]:
# These list can be iterated throgh for loop
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
# -------------------------------------------------------------------------------
# this part is to define input data shape and confusion matrix
# -------------------------------------------------------------------------------
for length in times:
    cvscores = []
    print('Now, start for ' + length)
    if length == '2s' :
        input_shape = (120, 8, 8)
    elif length == '1s':
        input_shape = (60, 8, 8)
    else:
        input_shape = (30, 8, 8)
# -------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
# -------------------------------------------------------------------------------       
    for i in range(1,11):
        train_data, train_olabel = load_random_subject(length, 'stationary', 'train', i)
        test_data, test_olabel = load_random_subject(length, 'holding', 'test', i) 
        print('Now, start the cv: ',i)
        model = cnn_2d(input_shape = input_shape, learning_rate = 1e-3)
        history = model.fit(train_data,train_olabel,epochs = 20, batch_size = 32,verbose = 2,shuffle = True,
                          validation_data=(test_data,test_olabel))
        pred = np.argmax(model.predict(test_data), axis = 1)
        real_label = onehot_to_label(test_olabel)
        scores = accuracy_score(real_label, pred) * 100
        print("The accuracy in this fold: %f"%scores)
        print("=============================")
        cvscores.append(scores)
        conf_mat = np.expand_dims(confusion_matrix(real_label, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
# -------------------------------------------------------------------------------
# this part is to plot acc, loss curve and print out the average accuracy
# -------------------------------------------------------------------------------
        # plot_acc(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (train stationary test holding) accuracy.png')
        # plt.show()
        # plot_loss(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (train stationary test holding) loss.png')
        # plt.show()
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
# -------------------------------------------------------------------------------
# this part is to plot confusion matrix
# -------------------------------------------------------------------------------   
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                  normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/confusion matrix/' + '2D_CNN' + ' (train stationary test holding) '+ ' inter_subject confusion_matrix.png')
plt.show()
# -------------------------------------------------------------------------------

In [ ]:
# Theses list can be iterated throgh for loop
#-------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#-------------------------------------------------------------------------------
# this part is to define input data shape and confusion matrix
#-------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    cvscores = []
    print('Now, start for ' + length)
    if length == '2s' :
        input_shape = (120, 8, 8)
    elif length == '1s':
        input_shape = (60, 8, 8)
    else:
        input_shape = (30, 8, 8)
#-------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#-------------------------------------------------------------------------------
    for i in range(1,11):
        train_data, train_olabel = load_random_subject(length, 'holding', 'train', i)
        test_data, test_olabel = load_random_subject(length, 'stationary', 'test', i) 
        print('Now, start the cv: ',i)
        model = cnn_2d(input_shape = input_shape, learning_rate = 1e-3)
        history = model.fit(train_data,train_olabel,epochs = 20, batch_size = 32,verbose = 2,shuffle = True,
                          validation_data=(test_data,test_olabel))
        pred = np.argmax(model.predict(test_data), axis = 1)
        real_label = onehot_to_label(test_olabel)
        scores = accuracy_score(real_label, pred) * 100
        print("The accuracy in this fold: %f"%scores)
        print("=============================")
        cvscores.append(scores)
        conf_mat = np.expand_dims(confusion_matrix(real_label, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
#-------------------------------------------------------------------------------
# this part is to plot acc, loss curve and print out the average accuracy
#-------------------------------------------------------------------------------
        # plot_acc(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (train holding test stationary.png')
        # plt.show()
        # plot_acc(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (train holding test stationary) loss.png')
        # plt.show()
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
#-------------------------------------------------------------------------------
# this part is to plot confusion matrix
#-------------------------------------------------------------------------------       
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                  normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/confusion matrix/' + '2D_CNN' + ' (train holding test stationary) '+ ' inter_subject confusion_matrix.png')
plt.show()
# -------------------------------------------------------------------------------

# Inter Subject (Combine)

In [ ]:
# Theses list can be iterated throgh for loop
# -------------------------------------------------------------------------------
times = ['2s', '1s', '0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
# -------------------------------------------------------------------------------
# this part is to define input data shap and confusion matrix
# -------------------------------------------------------------------------------
s_conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    s_cvscores = []
    print('Now, start for ' + length)
    if length == '2s' :
        input_shape = (120, 8, 8)
    elif length == '1s':
        input_shape = (60, 8, 8)
    else:
        input_shape = (30, 8, 8)
# -------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
# -------------------------------------------------------------------------------
    for i in range(1,11):
        s_train_data, s_train_olabel = load_random_subject(length, 'stationary', 'train',  i)
        h_train_data, h_train_olabel = load_random_subject(length, 'holding', 'train', i)
        half_s_train_data, half_s_train_olabel, half_h_train_data, half_h_train_olabel = random_select(s_train_data, s_train_olabel, h_train_data, h_train_olabel)
        t_train_data = np.concatenate((half_s_train_data, half_h_train_data))
        t_train_olabel = np.concatenate((half_s_train_olabel, half_h_train_olabel))
        s_test_data, s_test_olabel = load_random_subject(length, 'stationary', 'test', i)
        print('Now, start the cv: ',i)
        model = cnn_2d(input_shape = input_shape, learning_rate = 1e-3)
        history = model.fit(t_train_data,t_train_olabel,epochs = 20, batch_size = 32,verbose = 2, shuffle = True,
                            validation_data = (s_test_data, s_test_olabel))
        s_pred = np.argmax(model.predict(s_test_data), axis = 1)
        s_real_label = onehot_to_label(s_test_olabel)
        s_scores = accuracy_score(s_real_label, s_pred) * 100
        print("The accuracy in this fold (test stationary): %f"%s_scores)
        print("=============================")
        s_cvscores.append(s_scores)
        s_conf_mat = np.expand_dims(confusion_matrix(s_real_label, s_pred), axis = 2)
        s_conf_mat_all = np.concatenate([s_conf_mat_all, s_conf_mat], axis = 2)
# -------------------------------------------------------------------------------
# this part is to plot acc, loss curve and print out the average accuracy
# -------------------------------------------------------------------------------  
        # plot_acc(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (combine test stationary).png')
        # plt.show()
        # plot_loss(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (combine test stationary) loss.png')
        # plt.show()
    print('Test : stationary')
    print(length + " Total Accuracy (average): ", np.mean(s_cvscores), np.std(s_cvscores))
    print("=============================================")
    print("=============================================") 
# -------------------------------------------------------------------------------
# this part is to plot confusion matrix
# -------------------------------------------------------------------------------   
s_conf_mat_sum = np.sum(s_conf_mat_all, axis = 2)
plot_confusion_matrix(s_conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                  normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/confusion matrix/' + '2D_CNN' + ' combine_half_test_stationary ' + ' inter_subject confusion_matrix.png')
plt.show()

In [ ]:
# Theses list can be iterated throgh for loop
#------------------------------------------------------------
times = ['2s', '1s', '0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
h_conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
#------------------------------------------------------------------------
# this part is to define input data shape and confusion matrix
# ----------------------------------------------------------------------
for length in times:
    h_cvscores = []
    print('Now, start for ' + length)
    if length == '2s' :
        input_shape = (120, 8, 8)
    elif length == '1s':
        input_shape = (60, 8, 8)
    else:
        input_shape = (30, 8, 8)
# -------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------         
    for i in range(1,11):
        s_train_data, s_train_olabel = load_random_subject(length, 'stationary', 'train',  i)
        h_train_data, h_train_olabel = load_random_subject(length, 'holding', 'train', i)
        half_s_train_data, half_s_train_olabel, half_h_train_data, half_h_train_olabel = random_select(s_train_data, s_train_olabel, h_train_data, h_train_olabel)
        t_train_data = np.concatenate((half_s_train_data, half_h_train_data))
        t_train_olabel = np.concatenate((half_s_train_olabel, half_h_train_olabel))
        h_test_data, h_test_olabel = load_random_subject(length, 'holding', 'test', i)
        print('Now, start the cv: ',i)
        model = cnn_2d(input_shape = input_shape, learning_rate = 1e-3)
        history = model.fit(t_train_data,t_train_olabel,epochs = 20, batch_size = 32,verbose = 2, shuffle = True,
                            validation_data = (h_test_data, h_test_olabel))
        h_pred = np.argmax(model.predict(h_test_data), axis = 1)
        h_real_label = onehot_to_label(h_test_olabel)
        h_scores = accuracy_score(h_real_label, h_pred) * 100
        print('The accuracy in this fold (test holding): %f'%h_scores)
        print("=============================")
        h_cvscores.append(h_scores)
        h_conf_mat = np.expand_dims(confusion_matrix(h_real_label, h_pred), axis = 2)
        h_conf_mat_all = np.concatenate([h_conf_mat_all, h_conf_mat], axis = 2)
# -------------------------------------------------------------------------------
# this part is to plot acc, loss curve and print out the average accuracy
# ------------------------------------------------------------------------------- 
        # plot_acc(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (combine test holding).png')
        # plt.show()
        # plot_loss(history)
        # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/acc and loss/' + '2D_CNN' + '_' + length + '_inter subject (combine test holding) loss.png')
        # plt.show()
    print('Test : holding')
    print(length + " Total Accuracy (average): ", np.mean(h_cvscores), np.std(h_cvscores))
    print("=============================================")
    print("=============================================")
# -------------------------------------------------------------------------------
# this part is to plot confusion matrix
# -------------------------------------------------------------------------------
h_conf_mat_sum = np.sum(h_conf_mat_all, axis = 2)
plot_confusion_matrix(h_conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                  normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/2D_CNN/confusion matrix/' + '2D_CNN' + ' combine_half_test_holding ' + ' inter_subject confusion_matrix.png')
plt.show()